In [1]:
import pandas as pd
import os
import codecs
from bs4 import BeautifulSoup
import numpy as np
from datetime import datetime


In [2]:
out_dir = './PROCESSED_DATA'
source ='./RAW_DATA/source.html'
sts_list = ['new','confirmed','updated','removed']
library ='./RAW_DATA/library.csv'
nav_x_path = '//nav/ul'

ldf = pd.read_csv(library,parse_dates = ['dateAdded'])
ldf['status'] = np.nan
ldf.head()


,name,path,level,dateAdded,status
0,test,test/test1,1.0,2018-12-29,NaN
1,Articles,/appian.com/wiki/articles?authuser=0,1.0,2018-12-29,NaN
2,Home,/appian.com/wiki/home?authuser=0,1.0,2019-01-03,NaN
3,ML Use-Case ?s,/appian.com/wiki/articles/ml-use-case-s?authus...,2.0,2019-04-21,NaN
4,Cross Env Code Mgmt,/appian.com/wiki/articles/cross-env-code-mgmt?...,2.0,2019-04-21,NaN


In [3]:
df = pd.DataFrame()
soup = BeautifulSoup(codecs.open(source, 'r').read(), 'html.parser')
test = soup.nav.ul.find_all('a')

for child in soup.nav.ul.find_all('a'):
    df = df.append({'path':child.get('href'),
                    'name_source':child.get_text() ,
                    'level_source':child.get('data-level')},
                    ignore_index=True)

df.head()

,level_source,name_source,path
0,1,Home,/appian.com/wiki/home?authuser=0
1,1,Articles,/appian.com/wiki/articles?authuser=0
2,2,ML Use-Case ?s,/appian.com/wiki/articles/ml-use-case-s?authus...
3,2,Cross Env Code Mgmt,/appian.com/wiki/articles/cross-env-code-mgmt?...
4,2,Appian Date & Datetime Usage,/appian.com/wiki/articles/appian-date-datetime...


In [4]:
join_df = pd.merge(ldf, df, how='outer', on='path')
join_df.head()

,name,path,level,dateAdded,status,level_source,name_source
0,test,test/test1,1.0,2018-12-29,NaN,NaN,NaN
1,Articles,/appian.com/wiki/articles?authuser=0,1.0,2018-12-29,NaN,1,Articles
2,Home,/appian.com/wiki/home?authuser=0,1.0,2019-01-03,NaN,1,Home
3,ML Use-Case ?s,/appian.com/wiki/articles/ml-use-case-s?authus...,2.0,2019-04-21,NaN,2,ML Use-Case ?s
4,Cross Env Code Mgmt,/appian.com/wiki/articles/cross-env-code-mgmt?...,2.0,2019-04-21,NaN,2,Cross Env Code Mgmt


In [5]:
# set removed status
join_df.loc[join_df.name_source.isnull(), 'status'] = sts_list[3]
join_df.head()

,name,path,level,dateAdded,status,level_source,name_source
0,test,test/test1,1.0,2018-12-29,removed,NaN,NaN
1,Articles,/appian.com/wiki/articles?authuser=0,1.0,2018-12-29,NaN,1,Articles
2,Home,/appian.com/wiki/home?authuser=0,1.0,2019-01-03,NaN,1,Home
3,ML Use-Case ?s,/appian.com/wiki/articles/ml-use-case-s?authus...,2.0,2019-04-21,NaN,2,ML Use-Case ?s
4,Cross Env Code Mgmt,/appian.com/wiki/articles/cross-env-code-mgmt?...,2.0,2019-04-21,NaN,2,Cross Env Code Mgmt


In [6]:
# set new status

join_df.loc[join_df.name.isnull(), ['status','dateAdded']] = (sts_list[0],datetime.today().date())
join_df.level.fillna(join_df.level_source, inplace=True)
join_df.level_source.fillna(join_df.level, inplace=True)
join_df.name.fillna(join_df.name_source, inplace=True)
join_df.level = join_df.level.astype(np.int64)
join_df.level_source = join_df.level_source.astype(np.int64)
join_df.head()

,name,path,level,dateAdded,status,level_source,name_source
0,test,test/test1,1,2018-12-29,removed,1,NaN
1,Articles,/appian.com/wiki/articles?authuser=0,1,2018-12-29,NaN,1,Articles
2,Home,/appian.com/wiki/home?authuser=0,1,2019-01-03,NaN,1,Home
3,ML Use-Case ?s,/appian.com/wiki/articles/ml-use-case-s?authus...,2,2019-04-21,NaN,2,ML Use-Case ?s
4,Cross Env Code Mgmt,/appian.com/wiki/articles/cross-env-code-mgmt?...,2,2019-04-21,NaN,2,Cross Env Code Mgmt


In [7]:
# set confirmed status
join_df.loc[(join_df.name == join_df.name_source) & 
            (join_df.level == join_df.level_source) & 
            (join_df.status != sts_list[0]), ['status']] = sts_list[1]

join_df.head()

,name,path,level,dateAdded,status,level_source,name_source
0,test,test/test1,1,2018-12-29,removed,1,NaN
1,Articles,/appian.com/wiki/articles?authuser=0,1,2018-12-29,confirmed,1,Articles
2,Home,/appian.com/wiki/home?authuser=0,1,2019-01-03,confirmed,1,Home
3,ML Use-Case ?s,/appian.com/wiki/articles/ml-use-case-s?authus...,2,2019-04-21,confirmed,2,ML Use-Case ?s
4,Cross Env Code Mgmt,/appian.com/wiki/articles/cross-env-code-mgmt?...,2,2019-04-21,confirmed,2,Cross Env Code Mgmt


In [8]:
# set updated status
join_df.loc[join_df.status.isnull(), ['name','level']] = (np.nan, np.nan)
join_df.loc[join_df.status.isnull(), 'status'] = sts_list[2]
join_df.level.fillna(join_df.level_source, inplace=True)
join_df.name.fillna(join_df.name_source, inplace=True)
join_df.head()

,name,path,level,dateAdded,status,level_source,name_source
0,test,test/test1,1.0,2018-12-29,removed,1,NaN
1,Articles,/appian.com/wiki/articles?authuser=0,1.0,2018-12-29,confirmed,1,Articles
2,Home,/appian.com/wiki/home?authuser=0,1.0,2019-01-03,confirmed,1,Home
3,ML Use-Case ?s,/appian.com/wiki/articles/ml-use-case-s?authus...,2.0,2019-04-21,confirmed,2,ML Use-Case ?s
4,Cross Env Code Mgmt,/appian.com/wiki/articles/cross-env-code-mgmt?...,2.0,2019-04-21,confirmed,2,Cross Env Code Mgmt


In [10]:
join_df = join_df.drop(columns=['level_source', 'name_source'])
join_df.head()

,name,path,level,dateAdded,status
0,test,test/test1,1.0,2018-12-29,removed
1,Articles,/appian.com/wiki/articles?authuser=0,1.0,2018-12-29,confirmed
2,Home,/appian.com/wiki/home?authuser=0,1.0,2019-01-03,confirmed
3,ML Use-Case ?s,/appian.com/wiki/articles/ml-use-case-s?authus...,2.0,2019-04-21,confirmed
4,Cross Env Code Mgmt,/appian.com/wiki/articles/cross-env-code-mgmt?...,2.0,2019-04-21,confirmed
